In [5]:
from math import isinf
import cobra
import escher
import json
import os
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt

In [6]:
model = cobra.io.read_sbml_model('iJO1366.xml')

In [7]:
model.summary()

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      17.6       h2o_e  45.6   BIOMASS_Ec_i...  0.982
nh4_e     10.6       co2_e  19.7
glc__D_e  10         h_e     9.03
pi_e       0.948
so4_e      0.248
k_e        0.192
fe2_e      0.0158
mg2_e      0.00852
ca2_e      0.00511
cl_e       0.00511
cu2_e      0.000697
mn2_e      0.000679
zn2_e      0.000335
ni2_e      0.000317
mobd_e     0.000127


In [8]:
# Como ajustar o bounds
# uma maneira facil para quem nao percebe nada de constraints
model.medium

{'EX_ca2_e': 1000.0,
 'EX_cbl1_e': 0.01,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [9]:
# outra/melhor forma, controalr directamenet a reaccao do exchange
# usar um loop para identificar exchange reactions e filtrar por lower bound not 0
for r in model.reactions:
    if 'EX_' in r.id and not r.lower_bound==0:
        print(r,r.lower_bound,r.upper_bound)

EX_ca2_e: ca2_e <=>  -1000.0 1000.0
EX_cbl1_e: cbl1_e <=>  -0.01 1000.0
EX_cl_e: cl_e <=>  -1000.0 1000.0
EX_co2_e: co2_e <=>  -1000.0 1000.0
EX_cobalt2_e: cobalt2_e <=>  -1000.0 1000.0
EX_cu2_e: cu2_e <=>  -1000.0 1000.0
EX_fe2_e: fe2_e <=>  -1000.0 1000.0
EX_fe3_e: fe3_e <=>  -1000.0 1000.0
EX_glc__D_e: glc__D_e <=>  -10.0 1000.0
EX_h_e: h_e <=>  -1000.0 1000.0
EX_h2o_e: h2o_e <=>  -1000.0 1000.0
EX_k_e: k_e <=>  -1000.0 1000.0
EX_mg2_e: mg2_e <=>  -1000.0 1000.0
EX_mn2_e: mn2_e <=>  -1000.0 1000.0
EX_mobd_e: mobd_e <=>  -1000.0 1000.0
EX_na1_e: na1_e <=>  -1000.0 1000.0
EX_nh4_e: nh4_e <=>  -1000.0 1000.0
EX_ni2_e: ni2_e <=>  -1000.0 1000.0
EX_o2_e: o2_e <=>  -1000.0 1000.0
EX_pi_e: pi_e <=>  -1000.0 1000.0
EX_sel_e: sel_e <=>  -1000.0 1000.0
EX_slnt_e: slnt_e <=>  -1000.0 1000.0
EX_so4_e: so4_e <=>  -1000.0 1000.0
EX_tungs_e: tungs_e <=>  -1000.0 1000.0
EX_zn2_e: zn2_e <=>  -1000.0 1000.0


In [10]:
# mudar bound da glucose e oxigenio de acordo com o ijo1366 paper para aerobic growth on glucose
model.reactions.EX_glc__D_e.lower_bound=-8
model.reactions.EX_o2_e.lower_bound=-18.5
for r in model.reactions:
    if 'EX_' in r.id and not r.lower_bound==0:
        print(r,r.lower_bound,r.upper_bound)
model.summary()

EX_ca2_e: ca2_e <=>  -1000.0 1000.0
EX_cbl1_e: cbl1_e <=>  -0.01 1000.0
EX_cl_e: cl_e <=>  -1000.0 1000.0
EX_co2_e: co2_e <=>  -1000.0 1000.0
EX_cobalt2_e: cobalt2_e <=>  -1000.0 1000.0
EX_cu2_e: cu2_e <=>  -1000.0 1000.0
EX_fe2_e: fe2_e <=>  -1000.0 1000.0
EX_fe3_e: fe3_e <=>  -1000.0 1000.0
EX_glc__D_e: glc__D_e <=>  -8 1000.0
EX_h_e: h_e <=>  -1000.0 1000.0
EX_h2o_e: h2o_e <=>  -1000.0 1000.0
EX_k_e: k_e <=>  -1000.0 1000.0
EX_mg2_e: mg2_e <=>  -1000.0 1000.0
EX_mn2_e: mn2_e <=>  -1000.0 1000.0
EX_mobd_e: mobd_e <=>  -1000.0 1000.0
EX_na1_e: na1_e <=>  -1000.0 1000.0
EX_nh4_e: nh4_e <=>  -1000.0 1000.0
EX_ni2_e: ni2_e <=>  -1000.0 1000.0
EX_o2_e: o2_e <=>  -18.5 1000.0
EX_pi_e: pi_e <=>  -1000.0 1000.0
EX_sel_e: sel_e <=>  -1000.0 1000.0
EX_slnt_e: slnt_e <=>  -1000.0 1000.0
EX_so4_e: so4_e <=>  -1000.0 1000.0
EX_tungs_e: tungs_e <=>  -1000.0 1000.0
EX_zn2_e: zn2_e <=>  -1000.0 1000.0
IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ------------------

In [11]:
# do "escher."to see differwnt avialble maps, and different types of getting maps
escher.list_cached_maps()
#escher.list_cached_models()

[{'organism': 'Escherichia coli', 'map_name': 'map_v2'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'map_v1'},
 {'organism': 'Escherichia coli', 'map_name': 'new_map'}]

In [12]:
#escher
b = escher.Builder(map_name='map_v1', local_host='https://escher.github.io')
b.display_in_notebook()

In [13]:
#ver qual e a biomassa que esta a ser usada
print(model.objective)

Maximize
-1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1 + 1.0*BIOMASS_Ec_iJO1366_core_53p95M


In [14]:
# mudar biomassa do core para WT
model_biomass = 'BIOMASS_Ec_iJO1366_WT_53p95M'
model.objective= model_biomass
print(model.objective)

Maximize
-1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M


In [15]:
# corre FBA e pFBA
fba_solution = model.optimize()
pfba_solution = cobra.flux_analysis.pfba(model)
print(model.objective)

Maximize
-1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M


In [16]:
# comparar pFBA andf FBA
# o objctivo do pFBA e minimizar a soma dos fluxos, mas para ver o biomass flux temos de ver directamente 
#o flux da reaccao da biomass
print('FBA Ovalue', fba_solution.objective_value, 'pFBA Ovalue', pfba_solution.objective_value)
print('FBA Biomass',fba_solution.fluxes.BIOMASS_Ec_iJO1366_WT_53p95M, 'pFBA biomass', pfba_solution.fluxes.BIOMASS_Ec_iJO1366_WT_53p95M)

FBA Ovalue 0.7865875166015222 pFBA Ovalue 557.2926358538784
FBA Biomass 0.7865875166015222 pFBA biomass 0.7865875166015222


In [17]:
# pintar WT no nosso mapa

# pintar fluxos no escher
# {} dicionario vazio
# escher
# fazer free draw das reaccoes

#b = escher.Builder(reaction_data={'FRD3':1,'SUCOAS':-1}, map_name='iJO1366.Central metabolism', local_host='https://escher.github.io')
#b.display_in_notebook()

# criar o dicionario para pintar a solucao num mapa
# o break ajuda a ver so a primeira linha
WT_sol1={}
for r in model.reactions:
    
    # prints e break para debug
    #print(r, pfba_solution.fluxes[r.id])
    #break
    WT_sol1[r.id]=pfba_solution.fluxes[r.id]
    
#para confirmar que esta correcto    
#WT_sol1

b = escher.Builder(reaction_data=WT_sol1, map_name='map_v1', local_host='https://escher.github.io')
b.display_in_notebook()

In [18]:
# pintar WT no iJO13666

# pintar fluxos no escher
# {} dicionario vazio
# escher
# fazer free draw das reaccoes

#b = escher.Builder(reaction_data={'FRD3':1,'SUCOAS':-1}, map_name='iJO1366.Central metabolism', local_host='https://escher.github.io')
#b.display_in_notebook()

# criar o dicionario para pintar a solucao num mapa
# o break ajuda a ver so a primeira linha
WT_sol1={}
for r in model.reactions:
    
    # prints e break para debug
    #print(r, pfba_solution.fluxes[r.id])
    #break
    WT_sol1[r.id]=pfba_solution.fluxes[r.id]
    
#para confirmar que esta correcto    
#WT_sol1

b = escher.Builder(reaction_data=WT_sol1, map_name='iJO1366.Central metabolism', local_host='https://escher.github.io')
b.display_in_notebook()

In [19]:
# declarar funcao para contsruir dicionar e pintar no escher
def draw_escher(solution,escher_map):
    WT_sol1={}
    for r in model.reactions:

        # prints e break para debug
        #print(r, pfba_solution.fluxes[r.id])
        #break
        WT_sol1[r.id]=solution.fluxes[r.id]

    #para confirmar que esta correcto    
    #WT_sol1

    b = escher.Builder(reaction_data=WT_sol1, map_name=escher_map, local_host='https://escher.github.io')
    return b

#draw_escher(solution).display_in_notebook()

In [20]:
# moma sim

moma_sol_WT=cobra.flux_analysis.moma(model=model,solution=pfba_solution,linear=True)

In [25]:
moma_sol_WT

,fluxes,reduced_costs
DM_4crsol_c,1.754090e-04,0.000000
DM_5drib_c,1.817017e-04,0.000000
DM_aacald_c,-1.054712e-15,0.000000
DM_amob_c,1.573175e-06,0.000000
DM_mththf_c,1.054027e-03,0.000000
...,...,...
ZN2abcpp,0.000000e+00,0.320942
ZN2t3pp,0.000000e+00,0.000000
ZN2tpp,2.548544e-04,0.000000
ZNabcpp,0.000000e+00,0.320942


In [26]:
moma_sol_WT.objective_value

-1.7258845262345277e-14

In [27]:
# verificar solucao a pata

error_sum=0
for r in model.reactions:
    
    # prints e break para debug
    #print(r, pfba_solution.fluxes[r.id])
    #break
    pfba_value=pfba_solution.fluxes[r.id]
    moma_value=moma_sol_WT.fluxes[r.id]
    #print(pfba_value, moma_value)
    error_sum = error_sum + (pfba_value-moma_value)**2
error_sum
    

3.986849244148705e-26

In [28]:
#variavel global para comparar o error numerico
E_VALUE = 1e-7

#fold_percentage_change 0 is knockout
def force_flux(fold_percentage_change, target_reaction, reference_solution):
    # alterer percentagem de fluxo numa reaccao especifica
    # percentagem com variavel
    #fold_percentage_change=4
    # reaccao a mudarcomo variavel
    #target_reaction='AKGDH'
    # delcarar a reference solution
    #reference_solution=pfba_solution

    model_reaction = model.reactions.get_by_id(target_reaction)
    # processo de verificar o flux actual da target reaction (target flux) e fazer o incremento de flux (fold change)
    target_flux = reference_solution.fluxes[target_reaction]
    prev_lb = model.reactions.get_by_id(target_reaction).lower_bound
    prev_ub = model.reactions.get_by_id(target_reaction).upper_bound
    if fold_percentage_change == 0:
        model.reactions.get_by_id(target_reaction).lower_bound = 0 
        model.reactions.get_by_id(target_reaction).upper_bound = 0
    elif target_flux > 0:
        target_flux = target_flux + target_flux*fold_percentage_change
        model.reactions.get_by_id(target_reaction).lower_bound = target_flux
    else:
        target_flux = target_flux - target_flux*fold_percentage_change
        model.reactions.get_by_id(target_reaction).upper_bound = target_flux
    print('reference flux change',reference_solution.fluxes[target_reaction], '->',target_flux)
    
    print(model_reaction, model_reaction.lower_bound, model_reaction.upper_bound)
    # restringir a target reaction ao novo flux com a fold change
    #model.reactions.get_by_id(target_reaction).lower_bound = target_flux
    moma_sol=cobra.flux_analysis.moma(model=model,solution=reference_solution,linear=True)
    model.reactions.get_by_id(target_reaction).lower_bound = prev_lb
    model.reactions.get_by_id(target_reaction).upper_bound = prev_ub
    print('error',moma_sol.objective_value, 'biomass flux',moma_sol.fluxes.BIOMASS_Ec_iJO1366_WT_53p95M)
    
    # checkar todas as reaccoes que mudaram o luxo devido a esta mudanca

    fold_change_impact={}
    for r in model.reactions:
        ref_value=reference_solution.fluxes[r.id]
        moma_value=moma_sol.fluxes[r.id]
        #print((ref_value - moma_value)**2, E_VALUE)
        if (ref_value - moma_value)**2 > E_VALUE:
            fold_change_impact[r.id] = (ref_value, moma_value)
    return fold_change_impact, moma_sol, target_flux

#print(fold_change_impact)
#draw_escher(moma_sol).display_in_notebook()

In [29]:
# testar a funcao de force flux para uma reacao e fazer print de todos as mudancas
fold_change_impact, moma_sol, target_flux = force_flux(0.1, 'AKGDH', pfba_solution)
print(fold_change_impact)

reference flux change 3.1971759566572424 -> 3.5168935523229665
AKGDH: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c 3.5168935523229665 1000.0
error 9.36412931661071 biomass flux 0.7788834821724886
{'BIOMASS_Ec_iJO1366_WT_53p95M': (0.7865875166015222, 0.7788834821724886), 'EX_co2_e': (15.832896802339356, 15.96077175043154), 'EX_g3pe_e': (0.0, 0.000741264216681272), 'EX_glc__D_e': (-8.0, -7.998813747330674), 'EX_h2s_e': (0.0, 0.0018893296993404418), 'EX_ins_e': (0.0, 0.0031528760900609853), 'EX_k_e': (-0.14606143595773666, -0.1446308738046094), 'EX_lys__L_e': (0.0, 0.002568894872292393), 'EX_nh4_e': (-8.26410790733264, -8.203321714641506), 'EX_o2_e': (-14.232453139698244, -14.389095098725079), 'EX_pi_e': (-0.7292814696669904, -0.7234078855702042), 'EX_succ_e': (0.0, 0.02824920095255179), 'ACACT1r': (0.24634741138684774, 0.24431141228004605), 'ACACT2r': (0.24634741138684774, 0.24431141228004605), 'ACACT3r': (0.24634741138684774, 0.24431141228004605), 'ACACT4r': (0.24634505162429793,

In [30]:
#draw 
draw_escher(moma_sol,'map_v1').display_in_notebook()

In [31]:
# todos os fluxos que mudaram de 0 para qualquer coisa
for r in fold_change_impact:
    if fold_change_impact[r][0] == 0 and not fold_change_impact[r][1] == 0:
        print(r, fold_change_impact[r])

EX_g3pe_e (0.0, 0.000741264216681272)
EX_h2s_e (0.0, 0.0018893296993404418)
EX_ins_e (0.0, 0.0031528760900609853)
EX_lys__L_e (0.0, 0.002568894872292393)
EX_succ_e (0.0, 0.02824920095255179)
ACALD (0.0, 0.12435808069542637)
ACS (0.0, 0.009179816696246457)
ADK3 (0.0, 0.0022051719988076812)
AGDC (0.0, 0.0004249083148989141)
ALDD2y (0.0, 0.011225510046433351)
CYSDS (0.0, 0.001857542853286251)
DRPA (0.0, 0.00038775946088216126)
EDA (0.0, 0.08675339336795762)
EDD (0.0, 0.08675339336795762)
FACOAL160t2pp (0.0, 0.0009345147843106277)
FACOAL161t2pp (0.0, 0.0007268551042875077)
FACOAL181t2pp (0.0, 0.0003746163781461753)
FTHFLi (0.0, 0.22633507810100453)
G3PEtex (0.0, -0.000741264216681272)
GLCP (0.0, 0.0011878619565093979)
GLUSy (0.0, 0.006165102191604983)
GLXCL (0.0, 0.0003473820330489469)
GLYCK2 (0.0, 0.0003473820330489469)
H2St1pp (0.0, 0.0018893296993404418)
H2Stex (0.0, -0.0018893296993404418)
INSt2pp_copy2 (0.0, -0.0031528760900609853)
INStex (0.0, -0.0031528760900609853)
LPLIPAL1E160pp (

In [32]:
# a solucao do moma, o erro. a reaccao ou fold change, o erro maior diz que tem maior impacto
moma_sol.objective_value

9.36412931661071

In [34]:
# correr a funcao para varias reaccoes ao mesmo tempo.
# percursossores para valien e leucina
reactions_array = [
                   'DHAD1', #test pyr to 3mob
                   'KARA1', #test pyr to 3mob
                   'ACLS',  #test pyr to 3mob
                   'VALtex','VALt2rpp','LEUtex','LEUt2rpp','ILEtex','ILEt2rpp','CADVtpp','LYStex','DAPabcpp','26DAHtex', #transport
                   'VPAMTr','VALTA','LEUTAi','OMCDC','IPMD','IPPMIa','IPPMIb','IPPS','MOHMT', #3mob fork
                   'OAADC','ASPTA','ASP1DC','ASPK','ASAD', #pyr to aspsa_c
                   'ILETA','DHAD2','KARA2','ACHBS','OBTFL','THRD_L','THRS','HSK','HSDy', #ile_L_p to aspsa_c
                   'DHDPS','DHDPRy','THDPS','SDPTA','SDPDS','DAPE','DAPDC','LADGMDH','UAAGDS', #aspsa_c to ugmd/lys_L_c
                  ]
folds = [
          0,
          -0.4,-0.3,-0.2,-0.1,
          0.1, 0.2, 0.3, 0.4
        ]
reaction_moma_error = {}
all_solutions = {}
for r in reactions_array:
    reaction_moma_error[r] = {}
    all_solutions[r] = {}
    for f in folds:
        fold_change_impact, moma_sol, target_flux = force_flux(f, r, pfba_solution)
        reaction_moma_error[r][f] = moma_sol.objective_value
        all_solutions[r][f] = moma_sol
    
#plt.plot(reaction_moma_error)

reference flux change 0.6682359856786237 -> 0.6682359856786237
DHAD1: 23dhmb_c --> 3mob_c + h2o_c 0 0
error 85.78826214945029 biomass flux 0.0
reference flux change 0.6682359856786237 -> 0.4009415914071742
DHAD1: 23dhmb_c --> 3mob_c + h2o_c 0.4009415914071742 1000.0
error 7.323990109478933e-14 biomass flux 0.786587516601523
reference flux change 0.6682359856786237 -> 0.46776518997503663
DHAD1: 23dhmb_c --> 3mob_c + h2o_c 0.46776518997503663 1000.0
error -3.8084193935165917e-14 biomass flux 0.7865875166015225
reference flux change 0.6682359856786237 -> 0.534588788542899
DHAD1: 23dhmb_c --> 3mob_c + h2o_c 0.534588788542899 1000.0
error -1.3904602504827205e-13 biomass flux 0.7865875166015209
reference flux change 0.6682359856786237 -> 0.6014123871107614
DHAD1: 23dhmb_c --> 3mob_c + h2o_c 0.6014123871107614 1000.0
error 3.8710207160346026e-13 biomass flux 0.7865875166015237
reference flux change 0.6682359856786237 -> 0.7350595842464861
DHAD1: 23dhmb_c --> 3mob_c + h2o_c 0.7350595842464861 

error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
LEUtex: leu__L_e <-- leu__L_p -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
LEUtex: leu__L_e <-- leu__L_p -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
LEUtex: leu__L_e <-- leu__L_p -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
LEUtex: leu__L_e <-- leu__L_p -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
LEUt2rpp: h_p + leu__L_p --> h_c + leu__L_c 0 0
error 2.8130764559400696e-14 biomass flux 0.7865875166015224
reference flux change 0.0 -> 0.0
LEUt2rpp: h_p + leu__L_p <-- h_c + leu__L_c -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
LEUt2rpp: h_p + leu__L_p <-- h_c + leu__L_c -1000.0 0.0
error -1.72588

error 3.192971722707115e-14 biomass flux 0.7865875166015222
reference flux change 0.0 -> 0.0
DAPabcpp: 26dap__M_p + atp_c + h2o_c --> 26dap__M_c + adp_c + h_c + pi_c 0.0 0.0
error 3.192971722707115e-14 biomass flux 0.7865875166015222
reference flux change 0.0 -> 0.0
DAPabcpp: 26dap__M_p + atp_c + h2o_c --> 26dap__M_c + adp_c + h_c + pi_c 0.0 0.0
error 3.192971722707115e-14 biomass flux 0.7865875166015222
reference flux change 0.0 -> 0.0
26DAHtex: 26dap__M_e --> 26dap__M_p 0 0
error 1.642153226817244e-13 biomass flux 0.7865875166015219
reference flux change 0.0 -> 0.0
26DAHtex: 26dap__M_e <-- 26dap__M_p -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
26DAHtex: 26dap__M_e <-- 26dap__M_p -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0
26DAHtex: 26dap__M_e <-- 26dap__M_p -1000.0 0.0
error -1.7258845262345277e-14 biomass flux 0.786587516601522
reference flux change 0.0 -> 0.0


error 82.76247264038993 biomass flux 0.0
reference flux change 0.34435070984278116 -> 0.2066104259056687
IPMD: 3c2hmp_c + nad_c --> 3c4mop_c + h_c + nadh_c 0.2066104259056687 1000.0
error -2.4106309293661237e-13 biomass flux 0.7865875166015268
reference flux change 0.34435070984278116 -> 0.2410454968899468
IPMD: 3c2hmp_c + nad_c --> 3c4mop_c + h_c + nadh_c 0.2410454968899468 1000.0
error -1.4189524101200078e-13 biomass flux 0.7865875166015212
reference flux change 0.34435070984278116 -> 0.2754805678742249
IPMD: 3c2hmp_c + nad_c --> 3c4mop_c + h_c + nadh_c 0.2754805678742249 1000.0
error -4.738358619203796e-14 biomass flux 0.7865875166015215
reference flux change 0.34435070984278116 -> 0.30991563885850304
IPMD: 3c2hmp_c + nad_c --> 3c4mop_c + h_c + nadh_c 0.30991563885850304 1000.0
error 4.85532739349825e-14 biomass flux 0.7865875166015226
reference flux change 0.34435070984278116 -> 0.37878578082705927
IPMD: 3c2hmp_c + nad_c --> 3c4mop_c + h_c + nadh_c 0.37878578082705927 1000.0
error 

cobra/util/solver.py:408 UserWarning: solver status is 'infeasible'


error -6.787073903688623e-13 biomass flux 0.7865875166015177
reference flux change 0.00045307440956232625 -> 0.0005436892914747915
MOHMT: 3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c 0.0005436892914747915 1000.0
error -1.3818727756282904e-12 biomass flux 0.7865875166015187
reference flux change 0.00045307440956232625 -> 0.0005889967324310241
MOHMT: 3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c 0.0005889967324310241 1000.0
error -4.831940035081121e-13 biomass flux 0.7865875166015166
reference flux change 0.00045307440956232625 -> 0.0006343041733872567
MOHMT: 3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c 0.0006343041733872567 1000.0
error -1.7001795622504418e-12 biomass flux 0.7865875166015236
reference flux change 0.0 -> 0.0
OAADC: h_c + oaa_c --> co2_c + pyr_c 0 0
error -2.3077428600889546e-13 biomass flux 0.7865875166015217
reference flux change 0.0 -> 0.0
OAADC: h_c + oaa_c --> co2_c + pyr_c 0.0 0.0
error -2.3077428600889546e-13 biomass flux 0.7865875166015217
reference flux change 0.0 ->

error -8.938937984273748e-15 biomass flux 0.7865875166015233
reference flux change -0.8228822377925497 -> -0.6583057902340397
ASAD: aspsa_c + nadp_c + pi_c <-- 4pasp_c + h_c + nadph_c -1000.0 -0.6583057902340397
error 2.1468031599228928e-13 biomass flux 0.7865875166015212
reference flux change -0.8228822377925497 -> -0.5760175664547847
ASAD: aspsa_c + nadp_c + pi_c <-- 4pasp_c + h_c + nadph_c -1000.0 -0.5760175664547847
error 1.0407560864881623e-13 biomass flux 0.7865875166015239
reference flux change -0.8228822377925497 -> -0.49372934267552976
ASAD: aspsa_c + nadp_c + pi_c <-- 4pasp_c + h_c + nadph_c -1000.0 -0.49372934267552976
error -1.6131344467157014e-13 biomass flux 0.7865875166015213
reference flux change -0.2220583754617093 -> -0.2220583754617093
ILETA: akg_c + ile__L_c --> 3mop_c + glu__L_c 0 0
error 81.37377291723843 biomass flux 0.0
reference flux change -0.2220583754617093 -> -0.31088172564639305
ILETA: akg_c + ile__L_c <-- 3mop_c + glu__L_c -1000.0 -0.31088172564639305
err

error -1.9666897442737483e-13 biomass flux 0.7865875166015186
reference flux change 0.0 -> 0.0
OBTFL: 2obut_c + coa_c --> for_c + ppcoa_c 0.0 0.0
error -1.9666897442737483e-13 biomass flux 0.7865875166015186
reference flux change 0.0 -> 0.0
OBTFL: 2obut_c + coa_c --> for_c + ppcoa_c 0.0 0.0
error -1.9666897442737483e-13 biomass flux 0.7865875166015186
reference flux change 0.0 -> 0.0
OBTFL: 2obut_c + coa_c --> for_c + ppcoa_c 0.0 0.0
error -1.9666897442737483e-13 biomass flux 0.7865875166015186
reference flux change 0.0 -> 0.0
OBTFL: 2obut_c + coa_c --> for_c + ppcoa_c 0.0 0.0
error -1.9666897442737483e-13 biomass flux 0.7865875166015186
reference flux change 0.2220583754617093 -> 0.2220583754617093
THRD_L: thr__L_c --> 2obut_c + nh4_c 0 0
error 81.37377291723857 biomass flux 0.0
reference flux change 0.2220583754617093 -> 0.13323502527702558
THRD_L: thr__L_c --> 2obut_c + nh4_c 0.13323502527702558 1000.0
error -7.64089864006918e-14 biomass flux 0.7865875166015205
reference flux change

error -8.031501459037188e-14 biomass flux 0.7865875166015245
reference flux change 0.2839777581810646 -> 0.31237553399917106
DHDPS: aspsa_c + pyr_c --> 23dhdp_c + 2.0 h2o_c + h_c 0.31237553399917106 1000.0
error 1.2858017333384555 biomass flux 0.7813174719205691
reference flux change 0.2839777581810646 -> 0.3407733098172775
DHDPS: aspsa_c + pyr_c --> 23dhdp_c + 2.0 h2o_c + h_c 0.3407733098172775 1000.0
error 2.5723546058687567 biomass flux 0.7760399044436456
reference flux change 0.2839777581810646 -> 0.3691710856353839
DHDPS: aspsa_c + pyr_c --> 23dhdp_c + 2.0 h2o_c + h_c 0.3691710856353839 1000.0
error 3.858907446778835 biomass flux 0.7707623369667196
reference flux change 0.2839777581810646 -> 0.3975688614534904
DHDPS: aspsa_c + pyr_c --> 23dhdp_c + 2.0 h2o_c + h_c 0.3975688614534904 1000.0
error 5.145460287688781 biomass flux 0.7654847694897743
reference flux change 0.2839777581810646 -> 0.2839777581810646
DHDPRy: 23dhdp_c + h_c + nadph_c --> nadp_c + thdp_c 0 0
error 83.4412720683

error -3.846332636208379e-14 biomass flux 0.7865875166015229
reference flux change 0.2839777581810646 -> 0.22718220654485166
DAPE: 26dap_LL_c --> 26dap__M_c 0.22718220654485166 1000.0
error 1.0322385399897402e-13 biomass flux 0.7865875166015218
reference flux change 0.2839777581810646 -> 0.2555799823629581
DAPE: 26dap_LL_c --> 26dap__M_c 0.2555799823629581 1000.0
error 2.9206950361566985e-15 biomass flux 0.7865875166015222
reference flux change 0.2839777581810646 -> 0.31237553399917106
DAPE: 26dap_LL_c --> 26dap__M_c 0.31237553399917106 1000.0
error 1.2858018914393827 biomass flux 0.7813174719205672
reference flux change 0.2839777581810646 -> 0.3407733098172775
DAPE: 26dap_LL_c --> 26dap__M_c 0.3407733098172775 1000.0
error 2.5723550066778222 biomass flux 0.776039904443644
reference flux change 0.2839777581810646 -> 0.3691710856353839
DAPE: 26dap_LL_c --> 26dap__M_c 0.3691710856353839 1000.0
error 3.858908048135613 biomass flux 0.7707623369667205
reference flux change 0.283977758181064

In [35]:
draw_escher(all_solutions['DHAD1'][0.2],'map_v1').display_in_notebook()

In [38]:
x_data = {}
for r in all_solutions:
    x_data[r] = {}
    for fold in all_solutions[r]:
        x_data[r][fold] = {}
        growth = all_solutions[r][fold].fluxes[model_biomass]
        if not growth == 0:
            rxn_change = {}
            top_changes = {}
            solution = all_solutions[r][fold]
            for rxn in model.reactions:
                #solution.fluxes.
                reference_flux = pfba_solution.fluxes[rxn.id]
                moma_flux = solution.fluxes[rxn.id]
                diff_flux = reference_flux - moma_flux
                
                #print(diff_flux)
                if abs(diff_flux) > E_VALUE:
                    change = 1 - (moma_flux / reference_flux)
                    rxn_change[rxn.id] = change
                    
                    if not isinf(change) and not change in top_changes:
                        top_changes[change] = []
                    if not isinf(change):
                        top_changes[change].append(rxn.id)
                    #print(r, f, rxn.id, reference_flux, moma_flux, diff_flux, change)
                    
            values = list(top_changes.keys())
            values.sort(reverse=True)
            for i in range(5):
                if len(values) > (i + 1):
                    #print('Rank', i,values[i])
                    for rxn_id in top_changes[values[i]]:
                        reference_flux = pfba_solution.fluxes[rxn_id]
                        moma_flux = solution.fluxes[rxn_id]
                        diff_flux = reference_flux - moma_flux
                        change = 1 - (moma_flux / reference_flux)
                        x_data[r][fold][rxn_id] = values[i]
                        #print(values[i], rxn_id, reference_flux, moma_flux, diff_flux, change)
            
            
            
            #print(r, fold , solution.fluxes[model_biomass])
            #break
    #break

/Users/jplfaria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19 RuntimeWarning: divide by zero encountered in double_scalars


In [37]:
import numpy as np
import pandas as pd


index = []
data_points = []
rxn_index = {}
i_value = 0
for target in x_data:
    for fold in x_data[target]:
        change_data = x_data[target][fold]
        for rxn_id in x_data[target][fold]:
            #print(rxn_id, change_data[rxn_id])
            if not rxn_id in rxn_index:
                rxn_index[rxn_id] = i_value
                i_value += 1
                index.append(rxn_id)
                
#print('index',len(index))
for r in index:
    data_points.append([])

k = 0
for target in x_data:
    for fold in x_data[target]:        
        solution_points = []
        for r in index:
            v = 0
            if r in x_data[target][fold]:
                 v = x_data[target][fold][r]
            #print(r, v)
            data_points[k].append(v)
        #print(len(solution_points))
        #data_points.append(solution_points)
        
        k += 1

pd = pd.DataFrame(data_points, index=index)
#print(rxn_index)
#print(x_data)
#figure(figsize=(24,12))
pd.plot(kind='bar', figsize=(24,12))

IndexError: list index out of range

In [45]:
len(all_solutions)
all_solutions['KARA1'].keys()
all_solutions.keys()

dict_keys(['DHAD1', 'KARA1', 'ACLS', 'VALtex', 'VALt2rpp', 'LEUtex', 'LEUt2rpp', 'ILEtex', 'ILEt2rpp', 'CADVtpp', 'LYStex', 'DAPabcpp', '26DAHtex', 'VPAMTr', 'VALTA', 'LEUTAi', 'OMCDC', 'IPMD', 'IPPMIa', 'IPPMIb', 'IPPS', 'MOHMT', 'OAADC', 'ASPTA', 'ASP1DC', 'ASPK', 'ASAD', 'ILETA', 'DHAD2', 'KARA2', 'ACHBS', 'OBTFL', 'THRD_L', 'THRS', 'HSK', 'HSDy', 'DHDPS', 'DHDPRy', 'THDPS', 'SDPTA', 'SDPDS', 'DAPE', 'DAPDC', 'LADGMDH', 'UAAGDS', 'WT'])

In [42]:
all_solutions['WT'] = {}

In [43]:
all_solutions['WT']['pFBA'] = pfba_solution

In [48]:
#data['target']['fold']['rxn'] = flux
def get_flux_dict(solution, model):
    flux_dict={}
    for r in model.reactions:
        flux_dict[r.id]=solution.fluxes[r.id]
    return flux_dict

data = {}
for target in all_solutions:
    if not target in data:
        data[target] = {}
    for fold in all_solutions[target]:
        solution = all_solutions[target][fold]
        flux_map = get_flux_dict(solution, model)
        data[target][fold] = flux_map

In [51]:
with open('./moma_fluxes.json', 'w') as f:
    json.dump(data, f)